#  Matching Catalogs by Proximity with NumCosmo

---
**License**

 Matching Catalogs by Proximity with NumCosmo

 Mon Dec 2 18:30:00 2024\
 Copyright  2024\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---
 matching_by_proximity\
 Copyright (C) 2024 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Libraries & Configs

In [1]:
import math
import sys

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
import pandas as pd

sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "/global/homes/c/cinlima/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/MachingCatalogs")

import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)


from numcosmo_py import Nc, Ncm, sky_match

Ncm.cfg_init()

import tqdm

# GCRCatalogs.get_available_catalog_names()

GCRCatalogs = 1.9.0 | GCR = 0.9.2


## Catalogs

In [2]:
#Cosmo DC2
cosmodc2= GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

#Cosmo DC2 + RedMaPPer
catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

In [3]:
for i in catalog_dc2_redmapper.list_all_quantities():
    if 'redshift' in i:
        print (i)

# print(', '.join(sorted(cosmodc2.list_all_quantities())))
# cosmodc2.get_catalog_info()

redshift_err
redshift_true_member
redshift_true_cg
redshift


In [4]:
# Restrictions on richness, mass and redshift
min_richness = 5
min_halo_mass = 1e13 #Msun
redshift_max = 1.2

#cDC2 + RM data
quantities = catalog_dc2_redmapper.list_all_quantities()
cluster_quantities = [q for q in quantities if 'member' not in q]
member_quantities = [q for q in quantities if 'member' in q]
    
query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')
cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities, [query])) # cDC2 + RM cluster data
member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

#cDC2 data
quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

In [5]:
table_halos = truth_data[truth_data['is_central'] == True]
# truth_data[truth_data['halo_id'] == 1915500193286]

In [6]:
# cluster_data[cluster_data['cluster_id'] == 41339]

## Matching by Proximity

In [7]:
area = 439.78987
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 71.0)
cosmo.param_set_by_name("Omegab", 0.0448)
cosmo.param_set_by_name("Omegac", 0.2648) #0.2603
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("w", -1.0) #-1.0


prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA",3.0116707481421923)
prim.param_set_by_name("n_SA", 0.963)


reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

halo_coordinates = {"RA":"ra" , "DEC":"dec" , "z":"redshift"}
halo_properties  = {"halo_mass":"mass"}

detections_coordinates =  {"RA":"ra" , "DEC":"dec" , "z":"redshift"}
detections_properties  = {"richness":"R" , "redshift_err":"z_err" , "richness_err":"R_err"}


halos = sky_match.SkyMatch(query_data=table_halos, query_coordinates=halo_coordinates,match_data=cluster_data,
                   match_coordinates=detections_coordinates)

detections = halos.invert_query_match()

In [8]:
halos_matched =  halos.match_2d(cosmo, 100 ,distance_method= sky_match.DistanceMethod.QUERY_RADIUS)
detections_matched =  detections.match_2d(cosmo, 100,distance_method= sky_match.DistanceMethod.MATCH_RADIUS)


mask_halos = halos_matched.filter_mask_by_redshift_proximity(sigma_z= 0.0, match_sigma_z_column='redshift_err') & halos_matched.filter_mask_by_distance(1)
mask_detections = detections_matched.filter_mask_by_redshift_proximity(sigma_z= 0.0, query_sigma_z_column='redshift_err') & detections_matched.filter_mask_by_distance(1)


halos_table = halos_matched.to_table_complete(query_properties=halo_properties , match_properties=detections_properties,mask=mask_halos)
detections_table = detections_matched.to_table_complete(query_properties=detections_properties , match_properties=halo_properties,mask=mask_detections)


unique_halos = halos_matched.select_best(mask=mask_halos , selection_criteria=sky_match.SelectionCriteria.DISTANCES)
unique_detections = detections_matched.select_best(mask=mask_detections , selection_criteria=sky_match.SelectionCriteria.MORE_MASSIVE , more_massive_column='halo_mass')

unique_halos_table = halos_matched.to_table_best(best=unique_halos , query_properties=halo_properties , match_properties=detections_properties)
unique_detections_table = detections_matched.to_table_best(best=unique_detections , query_properties=detections_properties , match_properties=halo_properties)

cross_indices = unique_halos.get_cross_match_indices(unique_detections)
cross = Table(names=(unique_halos_table.columns) , dtype=tuple([unique_halos_table.columns[i].dtype for i in range(len(unique_halos_table.columns))]))
for halo in cross_indices:
    cross_halo = unique_halos_table[unique_halos_table['ID'] == halo]
    cross.add_row(cross_halo[0])


In [10]:
# cross.write('match_proximity.fits', format='fits')
